In [500]:
import pandas as pd
import warnings
import requests
from bs4 import BeautifulSoup
import re
warnings.filterwarnings('ignore')

# Import merged csv of raw data. Data courtesy of baseball-reference.com
df = pd.read_csv (r'/Users/lochlanhendrix/Desktop/TripleAHitters.csv')

#Add column for what team a player is on
team = []
for i in range (0, 39):
    team.append("El Paso")
for i in range (39, 62):
    team.append("Oklahoma City")
for i in range (62,98):
    team.append("Round Rock")
for i in range (98,127):
    team.append("Sugar Land")
for i in range (127, 155):
    team.append("Albuquerque")
for i in range (155, 192):
    team.append("Reno")
for i in range (192, 237):
    team.append("Tacoma")
for i in range (237, 279):
    team.append("Las Vegas")
for i in range (279, 315):
    team.append("Salt Lake")
for i in range (315, 362):
    team.append("Sacremento")   
for i in range (362, 396):
    team.append("Durham")
for i in range (396,425):
    team.append("Scranton")
for i in range (425,465):
    team.append("Jacksonville")
for i in range (465, 501):
    team.append("Buffalo")
for i in range (501, 542):
    team.append("Lehigh Valley")    
for i in range (542, 573):
    team.append("Worcester") 
for i in range (573, 614):
    team.append("Norfolk")
for i in range (614, 649):
    team.append("Rochester")  
for i in range (649, 689):
    team.append("Syracuse") 
for i in range (689, 724):
    team.append("Charlotte")  
for i in range (724, 760):
    team.append("Nashville")  
for i in range (760, 793):
    team.append ("Toledo")   
for i in range (793, 831):
    team.append("Columbus")   
for i in range (831, 871):
    team.append("Indianapolis")  
for i in range (871, 915):
    team.append("St. Paul")   
for i in range (915, 946):
    team.append("Memphis")  
for i in range (946, 978):
    team.append("Omaha")   
for i in range (978, 1011):
    team.append("Gwinnett") 
for i in range (1011, 1044):
    team.append("Iowa")
for i in range (1044, 1091):
    team.append("Louisville")    
    
df['Team'] = team


#Add column for what league a player is in
league = []
for i in range (0, 362 ):
    league.append("Pacific Coast League")
for i in range (362, 1091):
    league.append("International League")
df['League'] = league



Name = []
Name = df["Name"]
Bats = []

#Raw data is formatted so that a player that bats left will be denoted as FirstName LastName*, and a switch will be
#FirstName LastName#. Remove symbols from end of name and instead add a column that denotes batting preference.
for i in range (0, len(Name) ):
    
    iLen = len(Name[i])
    
    if (Name[i][iLen - 1] == '*'):
        newName = Name[i].replace('*','')
        Name[i] = newName
        Bats.append("Left")
        
    elif (Name[i][iLen - 1] == '#'):
        newName = Name[i].replace('#','')
        Name[i] = newName
        Bats.append("Switch")
        
    else:
        Bats.append("Right")
        
df['Name'] = Name
df['Bats'] = Bats

#Remove junk columns
df.drop('Rk', inplace = True, axis = 1)
df.drop('Name-additional', inplace = True, axis = 1)
df.drop('Notes', inplace = True, axis = 1)

#Export formatted data
#Dataset represents players that are currently in the minor leagues whose future major league success will be predicted
df.to_csv (r'/Users/lochlanhendrix/Desktop/TripleA_DF.csv', index = None, header=True) 

#Also want major league data, pull from ESPN
#I suck at this, credit to https://www.codecademy.com/resources/blog/web-scraping-python-beautiful-soup-mlb-stats/

url = 'https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2021/start/1'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
header = soup.find('tr', attrs = {'class': 'colhead'})
columns = [col.get_text() for col in header.find_all('td')]
newDF = pd.DataFrame(columns = columns)

for i in range(1,300,50):
    url = 'https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2021/start/{}'.format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    players = soup.find_all('tr', attrs = {'class':re.compile('row player-10-')})
    
    for player in players:
        stats = [stat.get_text() for stat in player.find_all('td')]
        tempDF = pd.DataFrame(stats).transpose()
        tempDF.columns = columns
        
        newDF = pd.concat([newDF,tempDF], ignore_index = True)

#Convert columns from string object to float

for i in range(2,len(columns)):
    colToChange = columns[i]
    newDF[colToChange] = pd.to_numeric(newDF[colToChange])
    
#Add column for OBP

OBP = []

for i in range (0,newDF.shape[0]):
    OBPi = (newDF.iloc[i,6] + newDF.iloc[i,11]) / (newDF.iloc[i,4] + newDF.iloc[i,11])
    OBP.append(OBPi)
    
newDF['OBP'] = OBP
    
    
    
#Export Major League dataset

newDF.to_csv (r'/Users/lochlanhendrix/Desktop/MajorLeagueDF.csv', index = None, header=True)

newDF[120:140]









,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA,OBP
120,,Avisail Garcia,10,135,461,68,121,18,0,29,86,38,121,8,4,0.262,0.318637
121,,Willy Adames,4,140,497,77,130,32,1,25,73,57,156,5,4,0.262,0.337545
122,,David Fletcher,4,157,626,74,164,27,3,2,47,31,60,15,3,0.262,0.296804
123,,Pete Alonso,3,152,561,81,147,27,3,37,94,60,127,3,0,0.262,0.333333
124,,Tommy Edman,3,160,641,91,168,41,3,11,56,38,95,30,5,0.262,0.303387
125,126,Evan Longoria,14,81,253,45,66,17,0,13,46,35,68,1,1,0.261,0.350694
126,,Phil Gosselin,9,104,345,40,90,14,0,7,47,24,81,4,2,0.261,0.308943
127,,Jesus Aguilar,8,131,449,49,117,23,0,22,93,46,93,0,0,0.261,0.329293
128,,Josh Bell,6,144,498,75,130,24,1,27,88,65,101,0,0,0.261,0.346359
129,,Jose Abreu,8,152,566,86,148,30,2,30,117,61,143,1,0,0.261,0.333333


In [498]:
#minorLeagueStatsdf = pd.read_csv (r'/Users/lochlanhendrix/Desktop/CorrelationDF.csv')
minorLeagueStatsdf[50:100]

,NAME,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP


In [522]:
#### import requests
from bs4 import BeautifulSoup
import urllib
from requests_html import HTML
from requests_html import HTMLSession
import re
import pandas as pd
import time

headers=['Name','G','PA','AB','R','H','2B','3B','HR','RBI','SB','CS','BB','SO','BA','OBP','SLG','OPS','TB','GDP','HBP','SH','SF','IBB']
finalDF = pd.DataFrame(columns = headers)


def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)
        
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.co.uk/search?q=" + query)

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links




# urlList=[]
# for i in range(newDF.shape[0]):
#     queryString = newDF['PLAYER'][i] + " AAA Stats"
#     linksList=scrape_google(queryString)
#     time.sleep(.1)
#     for j in range(0,len(linksList)):
#         if 'https://www.baseball-reference.com/register/player.fcgi' in linksList[j]:
#             urlList.append(linksList[j])
#             break

    
    

for i in range (0, len(urlList)):   
    url=urlList[i]
    html=requests.get(url)
    time.sleep(5)
    content=html.content
    encoding='utf-8'
    htmlString = str(content,encoding)
    soup = BeautifulSoup(content,"html.parser")
    playersName = soup.find("strong").get_text()
    


    start = htmlString.find('>AAA (')
    end = htmlString.find('>AA (')
    relevantHtml = htmlString[start:end]
    


    newSoup = BeautifulSoup(relevantHtml,"html.parser")
    numbers = newSoup.find_all("td")
    stats=[]
    stats.append(playersName)

    
    for text in numbers:
        stats.append(text.get_text())
        
    del stats[1:4]

    
    
        

    try:
        finalDF.loc[len(finalDF)] = stats
        print("test " + str(i) + " worked")
    
    except:
        print("test "+ str(i) + " did not work")
        print(urlList[i])
        
        






finalDF








test 0 worked
test 1 worked
test 2 worked
test 3 worked
test 4 worked
test 5 did not work
https://www.baseball-reference.com/register/player.fcgi?id=soto--003jua
test 6 worked
test 7 worked
test 8 worked
test 9 worked
test 10 did not work
https://www.baseball-reference.com/register/player.fcgi?id=castel002nic
test 11 worked
test 12 worked
test 13 worked
test 14 worked
test 15 worked
test 16 worked
test 17 did not work
https://www.baseball-reference.com/register/player.fcgi?id=posey-001bus
test 18 worked
test 19 worked
test 20 worked
test 21 worked
test 22 worked
test 23 worked
test 24 worked
test 25 worked
test 26 worked
test 27 did not work
https://www.baseball-reference.com/register/player.fcgi?id=goldsc001pau
test 28 worked
test 29 worked
test 30 worked
test 31 worked
test 32 worked
test 33 worked
test 34 worked
test 35 worked
test 36 worked
test 37 did not work
https://www.baseball-reference.com/register/player.fcgi?id=franco002wan
test 38 worked
test 39 worked
test 40 worked
test 

,Name,G,PA,AB,R,H,2B,3B,HR,RBI,...,BA,OBP,SLG,OPS,TB,GDP,HBP,SH,SF,IBB
0,Luis Robert,56,264,236,48,69,11,5,17,42,...,.292,.349,.598,.946,141,6,7,0,5,0
1,Trea Turner,134,586,528,92,159,29,11,9,48,...,.301,.360,.449,.809,237,5,0,2,5,1
2,Frank Schwindel,344,1440,1348,194,397,94,1,69,261,...,.295,.331,.519,.851,700,41,12,0,12,1
3,Yuli Gurriel,4,19,18,3,4,1,0,1,2,...,.222,.263,.444,.708,8,0,0,0,0,0
4,Ketel Marte,165,760,693,126,220,42,9,11,83,...,.318,.367,.452,.819,313,10,2,3,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,Jon Berti,164,602,528,79,115,14,5,10,47,...,.218,.307,.320,.627,169,14,6,2,3,0
273,David Dahl,171,710,649,109,196,48,7,22,107,...,.302,.353,.499,.852,324,10,2,1,6,1
274,Matt Chapman,67,289,251,44,60,7,3,23,43,...,.239,.329,.566,.894,142,4,1,0,3,2
275,Jake Fraley,63,267,233,44,66,13,3,12,44,...,.283,.363,.519,.883,121,1,4,0,3,1


In [523]:
finalDF.to_csv (r'/Users/lochlanhendrix/Desktop/MILBofMLBData.csv', header=True)




0          Luis Robert
1          Trea Turner
2      Frank Schwindel
3         Yuli Gurriel
4          Ketel Marte
            ...       
295          Jon Berti
296         David Dahl
297       Matt Chapman
298        Jake Fraley
299         Brock Holt
Name: PLAYER, Length: 274, dtype: object

Luis Robert
